
## Extracting Named Entities (evaluation) <br>

We use named entities (NEs) extracted from both translated title and translated body text of articles in the evaluation dataset.

The extraction uses [spaCy API](https://spacy.io/).


### Extracting NEs from translated titles <br>

As one of our features, we extract named entities from the translated title using [spacy.io](https://spacy.io/) and count the common ones. However, there are a few problems with title NE recognition:

>**Issue 1**. 
spaCy relies heavily on capitalization, so the first word will not be recognised:
eg. the first word in _"Taoiseach makes first general election campaign pitch in New Year video"_ will not be extracted

>**Solution**:
Adding "In fact," at the beginning of every title. However, if the first word of the title is not a NE, the capitalization will lead to spaCy recognising it as NE, but we chose to have an excess of NE than not have important NE.
---------------------------
>**Issue 2.**
Some sentences have all words capitalized (except prepositions and conjunctions), eg. "Zomato Buys Uber's Food Delivery Business in India in an All-Stock Deal", which leads to spaCy extracting _"Zomato Buys Uber's"_, _'Food Delivery Business'_, _'India'_ as spaCy relies heavily on capitalization

>**Solution:**
Truecasing with **standfordNLP's stanza** tool to truecase the sentence. However, from _"In fact, Zomato buys Uber's Food Delivery Business in India in an all - stock deal"_, spaCy still can only extract "India", not to mention running truecasing for every sentence (even the one already truecased) can lead to all nouns being capitalized, which affects spaCy greatly, and since when there's no NE (eg. "Warm For The Season For Now, Some Wet Snow This Weekend") no unwanted NE is extracted (we chose NOT to extract certain entities like 'CARDINAL','DATE', 'MONEY', 'ORDINAL', 'PERCENT', 'QUANTITY', 'TIME') , so we ended up decide not to truecase as we could gain very little while creating a new issues.
-------------------------
>**Issue 3.**
The extracted NE needs to be standardized in order to be compared

>**Solution:**
We lowercase them and removed "'s", "."("US" and "U.S.") and "-".
    

In [7]:
import pandas as pd
import spacy #pip install spacy
import stanza
# pip install stanza
stanza.download('en')
nlp = spacy.load('en_core_web_sm') 


2022-03-17 15:48:01 INFO: Downloading default packages for language: en (English)...


2022-03-17 15:48:04 INFO: File exists: /Users/chung-fantsai/stanza_resources/en/default.zip.
2022-03-17 15:48:10 INFO: Finished downloading models and saved to /Users/chung-fantsai/stanza_resources.


In [6]:
title_df = pd.read_csv ('eval/_EVAL_title_translated.csv')
title_df

,pair_id,url1_lang,google_lang1,url2_lang,google_lang2,title1,title2,translated_title1,translated_title2
0,1484189203_1484121193,en,en,en,en,Police: 2 men stole tools from Lowe’s in Davie,No-swim advisory lifted for Deerfield Beach Pier,Police: 2 men stole tools from Lowe’s in Davie,No-swim advisory lifted for Deerfield Beach Pier
1,1484011097_1484011106,en,en,en,en,"Open database leaked 179GB in customer, US gov...",Best Western’s Massive Data Leak: 179GB Amazon...,"Open database leaked 179GB in customer, US gov...",Best Western’s Massive Data Leak: 179GB Amazon...
2,1484039488_1484261803,en,en,en,en,Ducks are own worst enemies in sloppy loss in ...,Woody Guthrie's 1943 New Year's Resolutions ar...,Ducks are own worst enemies in sloppy loss in ...,Woody Guthrie's 1943 New Year's Resolutions ar...
3,1484332324_1484796748,en,en,en,en,Another Bengal vs Centre tussle? Govt rejects ...,'Congress Rejected 7 Times': BJP's Reminder as...,Another Bengal vs Centre tussle? Govt rejects ...,'Congress Rejected 7 Times': BJP's Reminder as...
4,1484012256_1484419682,en,en,en,en,Bars and clubs you loved and lost this decade ...,Top 20 films of the 2010s,Bars and clubs you loved and lost this decade ...,Top 20 films of the 2010s
...,...,...,...,...,...,...,...,...,...
4897,1553907621_1553488848,es,es,it,it,Denver Nuggets reporta que “un miembro de la o...,"Coronavirus, un caso anche fra i Denver Nuggets","Denver Nuggets reports that ""a member of the o...","Coronavirus, a case also among the Denver Nuggets"
4898,1646957948_1643667075,es,es,it,it,Vivir en España es más barato que en la media ...,"Coronavirus, in Europa 140mila morti in più in...",Living in Spain is cheaper than in the Europea...,"Coronavirus, in Europe 140 thousand dead in Ma..."
4899,1504063453_1502866628,es,es,it,it,Activan sistema de vigilancia epidemiológica e...,Coronavirus in Cina: consumo di serpenti e zup...,Activate epidemiological surveillance system i...,Coronavirus in China: consumption of snakes an...
4900,1647862428_1647712939,es,es,it,it,Emite Irán orden de arresto contra Tump por as...,Iran emette mandato di arresto per Trump per l...,Emite Iran Arrest Order against Tump by Genera...,Iran emits stop mandate for trump for the murd...


 The chosen classes of named entities exclude the ones like _"CARDINAL"_, _"DATE"_, _"MONEY"_, _"ORDINAL"_, _"PERCENT"_, _"QUANTITY"_, and _"TIME"_. <br><br>

The reason for this is that the normalization of NEs in these categories might prove problematic. For instance, understanding that _"20/3/22"_ and _"the previous Sunday"_ correspond to the same date is not obvious. Moreover, we agreed that named entities, such as the ones referring to proper nouns of people or places, might be enough to capture the basic data of the story.

In [8]:

ignore_labels = ['CARDINAL','DATE', 'MONEY', 'ORDINAL', 'PERCENT', 'QUANTITY', 'TIME']

def overlapScore(key1, key2):
    key1 = set(key1)
    key2 = set(key2)
    interset = key1.intersection(key2)
    union = key1.union(key2)
    
    denominator = len(key1)
    if len(key2) < denominator:
        denominator = len(key2)
    
    if len(key1) <1 or len(key2) <1 or len(union) < 1 or len(interset)<1:
        return 0
    return (len(interset)/(denominator))

title_key_df = pd.DataFrame(columns = ["pair_id","url1_lang", "google_lang1","url2_lang","google_lang2", "translated_title1","translated_title2","key1","key2", "key_score"])

for index, row in title_df.iterrows(): 
    issue1_solve = 'In fact, '
    title1 = issue1_solve + row['translated_title1']
    title2 = issue1_solve + row['translated_title2']
    doc1 = nlp(title1) 
    doc2 = nlp(title2) 
    ner1 = [(((ent.text.lower()).replace("'s", "")).replace("-"," ")).replace(".","") for ent in doc1.ents if ent.label_ not in ignore_labels]
    ner2 = [(((ent.text.lower()).replace("'s", "")).replace("-"," ")).replace(".","") for ent in doc2.ents if ent.label_ not in ignore_labels]
    score = overlapScore(ner1, ner2)
    pair = row['pair_id']
    if score >0:
        print('---------------------',index,'-----------------------')
        print(pair)
        print(title1)
        print(ner1)
        print(title2)
        print(ner2)
        print(score)
    
    entry = {"pair_id": pair,"url1_lang":row['url1_lang'],"google_lang1":row['google_lang1'],"url2_lang":row['url2_lang'],"google_lang2":row['google_lang2'],"translated_title1":row["translated_title1"],"translated_title2":row["translated_title2"],"key1":ner1,"key2":ner2, "key_score":score}
    title_key_df = title_key_df.append(entry, ignore_index = True)
    

--------------------- 21 -----------------------
1484190460_1483806119
In fact, Telegram release a major update for the new year (changelog)
['telegram']
In fact, Telegram adds bunch of features including new Theme Editor, Send When Online, Podcast support and much more- Technology News, Firstpost
['telegram', 'theme editor', 'online', 'technology news']
1.0
--------------------- 49 -----------------------
1484037143_1484201620
In fact, Jahanbakhsh nets overhead kick as Brighton holds Chelsea 1-1
['jahanbakhsh', 'brighton', 'chelsea 1 1']
In fact, USMNT's Christian Pulisic back in Chelsea XI for 1-1 draw away to Brighton
['usmnt', 'christian pulisic', 'chelsea', 'brighton']
0.3333333333333333
--------------------- 53 -----------------------
1483806302_1483770632
In fact, Rep. John Lewis Of Atlanta Battling Stage 4 Pancreatic Cancer
['john lewis', 'atlanta battling stage 4 pancreatic cancer']
In fact, John Lewis represents an entire generation America can't afford to lose
['john lewis',

--------------------- 217 -----------------------
1562483638_1611624784
In fact, Zelensky voiced three scenarios with coronavirus in Ukraine.Video
['zelensky', 'ukraine']
In fact, Mortgage under 10%: Zelensky announced cheap loans for Ukrainians
['mortgage', 'zelensky', 'ukrainians']
0.5
--------------------- 222 -----------------------
1642220033_1618977537
In fact, In Krasnodar, found the missing Sherzod of Igamediev
['krasnodar', 'sherzod of igamediev']
In fact, In an opponent's opposite: In Krasnodar, the police are looking for a malicious violator of traffic rules
['krasnodar']
1.0
--------------------- 223 -----------------------
1609501071_1609409437
In fact, In the Ministry of Defense of the Russian Federation, they told how preparing for the Army 2020
['the ministry of defense', 'the russian federation', 'army']
In fact, The Ministry of Defense announced the most ambitious exercises in 2020
['the ministry of defense']
1.0
--------------------- 235 -----------------------
16253

--------------------- 321 -----------------------
1605048410_1594000116
In fact, Council's ceasefire agreement compromise China attitude into key
['council', 'china']
In fact, China-US contradictions do not solve the United Nations draft aunsence
['china', 'us', 'united nations']
0.5
--------------------- 334 -----------------------
1643509381_1619126052
In fact, Lin Zheng Yue: Hong Kong is still the ideal place for multinational corporate development after the implementation of national security law
['lin zheng yue', 'hong kong']
In fact, Lin Zheng Yue: Welcomes the decision of the National Congress through the establishment of a legal system and implementation mechanism for maintaining national security in Hong Kong SAR
['lin zheng yue', 'the national congress', 'hong kong']
1.0
--------------------- 335 -----------------------
1632977953_1630872412
In fact, Taiwan is surprisingly surprising to the US plane Beijing protests illegal
['taiwan', 'us', 'beijing']
In fact, Taiwan National

--------------------- 503 -----------------------
1514562858_1538268930
In fact, Consumer prices in Brazil go up in January, but less than expected
['brazil']
In fact, GDP of Brazil loses strength in fourth quarter but close 2019 with progress of 1.1%
['brazil']
1.0
--------------------- 504 -----------------------
1611623641_1614996201
In fact, Pillad two men disguised as Boya swimming in the sea in full confinement
['boya']
In fact, Pillad two men disguised as Boya to swim at sea during confinement
['boya']
1.0
--------------------- 509 -----------------------
1504781389_1504973190
In fact, Ministry of Health reported on the situation with Coronavirus near the border of Russia
['ministry of health', 'coronavirus', 'russia']
In fact, Moscow Department: Coronavirus has no found from China from China
['moscow department', 'coronavirus', 'china', 'china']
0.3333333333333333
--------------------- 512 -----------------------
1636095223_1637064680
In fact, What kind of conflict between poli

--------------------- 583 -----------------------
1646745400_1646298579
In fact, Medvedev voted by amendments to the Constitution without mask and gloves
['medvedev', 'constitution']
In fact, Medvedev urged the youth of the Russian Federation to participate in the voting on the Constitution
['medvedev', 'the russian federation', 'the constitution']
0.5
--------------------- 585 -----------------------
1527110798_1623237051
In fact, In Chechnya will open a monument to the dead health workers
['chechnya']
In fact, In Chechnya, the mosques will open and will introduce funerals for rites
['chechnya', 'mosques']
1.0
--------------------- 588 -----------------------
1497287604_1497506528
In fact, Under Kiev, a man attacks people: the inhabitant of Buchie with an iron hook broke his head
['kiev', 'buchie']
In fact, Under Kiev, an inadequate man attacks people (photo)
['kiev']
1.0
--------------------- 591 -----------------------
1629620910_1606103662
In fact, Anabolics, transported under the 

...

--------------------- 4506 -----------------------
1531972652_1529880359
In fact, 22 studies agree: 'Medicare for All' saves money
['medicare']
In fact, Here's What 22 Separate Studies Found: Medicare for All Would Cost Less Than the For-Profit Status Quo
['medicare']
1.0
--------------------- 4508 -----------------------
1592091128_1591556033
In fact, Cheryl's show The Greatest Dancer axed by BBC
['cheryl', 'the greatest dancer', 'bbc']
In fact, BBC cancels The Greatest Dancer after two series
['bbc', 'the greatest dancer']
1.0
--------------------- 4509 -----------------------
1571274943_1576594517
In fact, Who is Dominic Raab, the man running the country in Boris Johnson’s absence?
['dominic raab', 'boris johnson’s']
In fact, Who is Dominic Raab?
['dominic raab']
1.0
--------------------- 4511 -----------------------
1557010702_1579667345
In fact, Tom Hanks’ wife Rita Wilson raps Naughty By Nature while in quarantine
['tom hanks', 'rita wilson', 'nature']
In fact, Tom Hanks Gives Ma

--------------------- 4609 -----------------------
1530933267_1530959859
In fact, Prosecutor's Office clarifies that there is no recall record against Mon Laverte by Carabineros
['prosecutor office', 'mon laverte']
In fact, Public Ministry clarifies that there is no complaint against Mon Laverte and that Carabineros has not requested its statement
['public ministry', 'mon laverte', 'carabineros']
0.5
--------------------- 4610 -----------------------
1625621666_1625520166
In fact, Summit between Espinoza and Valenzuela: "This virus does not understand avenues, or streets that divide us"
['espinoza']
In fact, Espinoza and Valenzuela coordinated strategies to optimize tasks "without partisisms"
['espinoza', 'valenzuela']
1.0
--------------------- 4612 -----------------------
1640484317_1637791958
In fact, Pandemic has knees to Panama: Registered 25 222 infected and 493 dead
['panama']
In fact, Virus does not have a compassion of Panama: they register 470 dead and 22 597 infected
['virus'

--------------------- 4715 -----------------------
1487725219_1485057505
In fact, Several damage is reported in Puerto Rico after Sismo
['puerto rico']
In fact, New earthquake, now of 4.5, remits to Puerto Rico
['puerto rico']
1.0
--------------------- 4716 -----------------------
1551240100_1547419464
In fact, Aislinn Derbez talks about the treatment he received from Victoria Ruffo when he was with Eugenio
['aislinn derbez', 'victoria ruffo']
In fact, Aislinn Derbez Manda Message to Victoria Ruffo
['victoria ruffo']
1.0
--------------------- 4719 -----------------------
1586981410_1584233290
In fact, Video: They captured a jellyfish swimming in the canals of Venice
['venice']
In fact, Jellyfish slid through the clean waters of Venice canals
['jellyfish', 'venice']
1.0
--------------------- 4722 -----------------------
1602161279_1601966801
In fact, A robot maintains the social distance in Singapore
['singapore']
In fact, 'Spot', the dog-robot that guarantees the distancing in Singapor

--------------------- 4793 -----------------------
1505428056_1504214905
In fact, 18 cases of new new pneumonia in Chongqing have a total of 75 cases of diagnosis - News - Global IC Trade Starts Here.
['chongqing']
In fact, Pneumonia epidemic situation in Chongqing new coronavirus infection on January 25, 2020 - China News Net
['chongqing', 'china news net']
1.0
--------------------- 4794 -----------------------
1606379213_1605424353
In fact, Cumulative cure cases of Kazakhstan new crown pneumonia reached 2476 cases
['kazakhstan']
In fact, 363 cases of cure cases in Kazakhstan new crown pneumonia
['kazakhstan']
1.0
--------------------- 4795 -----------------------
1535894441_1535843341
In fact, Hijacking incident in Manila, Philippines, is trapped
['hijacking', 'manila', 'philippines']
In fact, The Philippines took a gunman's hosted personality incident about 30 people trapped shopping malls
['philippines']
1.0
--------------------- 4801 -----------------------
1489641413_1489827732
I

--------------------- 4892 -----------------------
1519429984_1518245422
In fact, Climb at 1,100 the number of dead by Coronavirus in China - so it's margarita
['coronavirus', 'china']
In fact, The Coronavirus is called Covid-19
['coronavirus']
1.0
--------------------- 4896 -----------------------
1529909197_1529885148
In fact, Weinstein, guilty of two sexual crimes in historical trial for methoo
['weinstein', 'methoo']
In fact, Harassment: Weinstein guilty of sexual violence and rape
['weinstein']
1.0


In [17]:
title_key_df

,pair_id,url1_lang,google_lang1,url2_lang,google_lang2,translated_title1,translated_title2,key1,key2,key_score
0,1484189203_1484121193,en,en,en,en,Police: 2 men stole tools from Lowe’s in Davie,No-swim advisory lifted for Deerfield Beach Pier,[lowe],[deerfield beach],0
1,1484011097_1484011106,en,en,en,en,"Open database leaked 179GB in customer, US gov...",Best Western’s Massive Data Leak: 179GB Amazon...,[us],[best western’s],0
2,1484039488_1484261803,en,en,en,en,Ducks are own worst enemies in sloppy loss in ...,Woody Guthrie's 1943 New Year's Resolutions ar...,[las vegas],"[woody guthrie, new year]",0
3,1484332324_1484796748,en,en,en,en,Another Bengal vs Centre tussle? Govt rejects ...,'Congress Rejected 7 Times': BJP's Reminder as...,[govt],"[congress rejected 7 times', bjp, maharashtra ...",0
4,1484012256_1484419682,en,en,en,en,Bars and clubs you loved and lost this decade ...,Top 20 films of the 2010s,[],[],0
...,...,...,...,...,...,...,...,...,...,...
4897,1553907621_1553488848,es,es,it,it,"Denver Nuggets reports that ""a member of the o...","Coronavirus, a case also among the Denver Nuggets",[denver nuggets],"[coronavirus, denver]",0
4898,1646957948_1643667075,es,es,it,it,Living in Spain is cheaper than in the Europea...,"Coronavirus, in Europe 140 thousand dead in Ma...","[spain, european]","[coronavirus, europe]",0
4899,1504063453_1502866628,es,es,it,it,Activate epidemiological surveillance system i...,Coronavirus in China: consumption of snakes an...,"[activate, venezuelan]","[coronavirus, china]",0
4900,1647862428_1647712939,es,es,it,it,Emite Iran Arrest Order against Tump by Genera...,Iran emits stop mandate for trump for the murd...,"[emite iran arrest order against, tump, soleim...",[iran],0


In [18]:
path = 'eval/_EVAL_title_named-entity_score.csv'
title_key_df.to_csv(path,index=False)

### Compare named entity similarity score <br>

We compare the annotated overall score of pairs with the similarity score obtained by 
overlap coefficient on the named entities extracted from the articles. <br><br>

This comparison is analogous to the one in the case of the training dataset, however, in principle, this step is not relevant for the final evaluation of our solution. This comparison can be thought as a double check to make sure that the choice of features makes sense and is motivated by curiosity rather then necessity.

In [13]:
import math

# see how indicating title named entities are to the similairty of two articles
eval_df = pd.read_csv ('eval/_EVAL_details_in_df.csv')

def normal_round(n):
    if n - math.floor(n) < 0.5:
        return int(math.floor(n))
    return int(math.ceil(n))

compare_df = pd.DataFrame(columns = ["overall","title_ne_score"])


for i, row in eval_df.iterrows(): 
    ## just to check our index order is still in place
#     if row["pair_id"] != title_key_df.iloc[i]['pair_id']:
#         print('---------------------',i,'-----------------------')
#         print(row["pair_id"])
    print('---------------------',i,'-----------------------')
    label = normal_round(row['Overall'])
    print('label:',label)
    score = title_key_df.iloc[i]['key_score']
    print('score:',score)
    
    entry = {"overall":label,"title_ne_score":score}
    compare_df = compare_df.append(entry, ignore_index = True)
    

--------------------- 0 -----------------------
label: 4
score: 0
--------------------- 1 -----------------------
label: 1
score: 0
--------------------- 2 -----------------------
label: 4
score: 0
--------------------- 3 -----------------------
label: 2
score: 0
--------------------- 4 -----------------------
label: 4
score: 0
--------------------- 5 -----------------------
label: 4
score: 0
--------------------- 6 -----------------------
label: 3
score: 0
--------------------- 7 -----------------------
label: 3
score: 0
--------------------- 8 -----------------------
label: 2
score: 0
--------------------- 9 -----------------------
label: 4
score: 0
--------------------- 10 -----------------------
label: 4
score: 0
--------------------- 11 -----------------------
label: 3
score: 0
--------------------- 12 -----------------------
label: 4
score: 0
--------------------- 13 -----------------------
label: 4
score: 0
--------------------- 14 -----------------------
label: 3
score: 0
-----

--------------------- 197 -----------------------
label: 3
score: 0
--------------------- 198 -----------------------
label: 1
score: 0
--------------------- 199 -----------------------
label: 3
score: 0
--------------------- 200 -----------------------
label: 3
score: 1.0
--------------------- 201 -----------------------
label: 4
score: 0
--------------------- 202 -----------------------
label: 3
score: 0
--------------------- 203 -----------------------
label: 1
score: 0
--------------------- 204 -----------------------
label: 3
score: 0
--------------------- 205 -----------------------
label: 2
score: 1.0
--------------------- 206 -----------------------
label: 4
score: 1.0
--------------------- 207 -----------------------
label: 3
score: 0
--------------------- 208 -----------------------
label: 2
score: 1.0
--------------------- 209 -----------------------
label: 2
score: 0
--------------------- 210 -----------------------
label: 4
score: 0
--------------------- 211 --------------

--------------------- 348 -----------------------
label: 1
score: 0
--------------------- 349 -----------------------
label: 4
score: 0
--------------------- 350 -----------------------
label: 3
score: 0
--------------------- 351 -----------------------
label: 4
score: 0
--------------------- 352 -----------------------
label: 1
score: 1.0
--------------------- 353 -----------------------
label: 4
score: 0
--------------------- 354 -----------------------
label: 4
score: 0
--------------------- 355 -----------------------
label: 4
score: 0
--------------------- 356 -----------------------
label: 1
score: 0
--------------------- 357 -----------------------
label: 3
score: 0
--------------------- 358 -----------------------
label: 3
score: 0
--------------------- 359 -----------------------
label: 4
score: 0
--------------------- 360 -----------------------
label: 2
score: 0
--------------------- 361 -----------------------
label: 4
score: 0
--------------------- 362 --------------------

--------------------- 509 -----------------------
label: 3
score: 0.3333333333333333
--------------------- 510 -----------------------
label: 3
score: 0
--------------------- 511 -----------------------
label: 2
score: 0
--------------------- 512 -----------------------
label: 1
score: 1.0
--------------------- 513 -----------------------
label: 4
score: 0
--------------------- 514 -----------------------
label: 3
score: 0.5
--------------------- 515 -----------------------
label: 4
score: 1.0
--------------------- 516 -----------------------
label: 4
score: 0
--------------------- 517 -----------------------
label: 4
score: 0
--------------------- 518 -----------------------
label: 1
score: 1.0
--------------------- 519 -----------------------
label: 4
score: 0
--------------------- 520 -----------------------
label: 3
score: 0
--------------------- 521 -----------------------
label: 4
score: 0
--------------------- 522 -----------------------
label: 3
score: 1.0
---------------------

label: 2
score: 1.0
--------------------- 677 -----------------------
label: 4
score: 0
--------------------- 678 -----------------------
label: 4
score: 0
--------------------- 679 -----------------------
label: 3
score: 0
--------------------- 680 -----------------------
label: 4
score: 0
--------------------- 681 -----------------------
label: 3
score: 0
--------------------- 682 -----------------------
label: 4
score: 0
--------------------- 683 -----------------------
label: 2
score: 0
--------------------- 684 -----------------------
label: 4
score: 0
--------------------- 685 -----------------------
label: 3
score: 1.0
--------------------- 686 -----------------------
label: 2
score: 0
--------------------- 687 -----------------------
label: 3
score: 0
--------------------- 688 -----------------------
label: 1
score: 0
--------------------- 689 -----------------------
label: 4
score: 1.0
--------------------- 690 -----------------------
label: 4
score: 1.0
--------------------- 

...

--------------------- 4149 -----------------------
label: 2
score: 0
--------------------- 4150 -----------------------
label: 1
score: 1.0
--------------------- 4151 -----------------------
label: 1
score: 0
--------------------- 4152 -----------------------
label: 3
score: 0
--------------------- 4153 -----------------------
label: 1
score: 1.0
--------------------- 4154 -----------------------
label: 1
score: 1.0
--------------------- 4155 -----------------------
label: 3
score: 1.0
--------------------- 4156 -----------------------
label: 1
score: 0
--------------------- 4157 -----------------------
label: 1
score: 1.0
--------------------- 4158 -----------------------
label: 2
score: 0
--------------------- 4159 -----------------------
label: 1
score: 1.0
--------------------- 4160 -----------------------
label: 2
score: 0
--------------------- 4161 -----------------------
label: 4
score: 1.0
--------------------- 4162 -----------------------
label: 2
score: 0
--------------------

--------------------- 4334 -----------------------
label: 1
score: 0
--------------------- 4335 -----------------------
label: 3
score: 0
--------------------- 4336 -----------------------
label: 1
score: 0
--------------------- 4337 -----------------------
label: 1
score: 1.0
--------------------- 4338 -----------------------
label: 2
score: 1.0
--------------------- 4339 -----------------------
label: 1
score: 0
--------------------- 4340 -----------------------
label: 1
score: 0
--------------------- 4341 -----------------------
label: 4
score: 0
--------------------- 4342 -----------------------
label: 2
score: 0
--------------------- 4343 -----------------------
label: 1
score: 0.6666666666666666
--------------------- 4344 -----------------------
label: 1
score: 0
--------------------- 4345 -----------------------
label: 1
score: 1.0
--------------------- 4346 -----------------------
label: 1
score: 0
--------------------- 4347 -----------------------
label: 1
score: 1.0
---------

--------------------- 4496 -----------------------
label: 2
score: 1.0
--------------------- 4497 -----------------------
label: 1
score: 1.0
--------------------- 4498 -----------------------
label: 1
score: 0
--------------------- 4499 -----------------------
label: 2
score: 0
--------------------- 4500 -----------------------
label: 4
score: 0
--------------------- 4501 -----------------------
label: 1
score: 1.0
--------------------- 4502 -----------------------
label: 2
score: 0
--------------------- 4503 -----------------------
label: 3
score: 0
--------------------- 4504 -----------------------
label: 1
score: 0.6666666666666666
--------------------- 4505 -----------------------
label: 3
score: 1.0
--------------------- 4506 -----------------------
label: 1
score: 1.0
--------------------- 4507 -----------------------
label: 1
score: 0
--------------------- 4508 -----------------------
label: 1
score: 1.0
--------------------- 4509 -----------------------
label: 1
score: 1.0
---

--------------------- 4636 -----------------------
label: 1
score: 1.0
--------------------- 4637 -----------------------
label: 2
score: 1.0
--------------------- 4638 -----------------------
label: 2
score: 1.0
--------------------- 4639 -----------------------
label: 2
score: 0
--------------------- 4640 -----------------------
label: 2
score: 0
--------------------- 4641 -----------------------
label: 1
score: 0
--------------------- 4642 -----------------------
label: 1
score: 0
--------------------- 4643 -----------------------
label: 1
score: 0
--------------------- 4644 -----------------------
label: 1
score: 0
--------------------- 4645 -----------------------
label: 2
score: 0
--------------------- 4646 -----------------------
label: 2
score: 1.0
--------------------- 4647 -----------------------
label: 1
score: 0
--------------------- 4648 -----------------------
label: 2
score: 0
--------------------- 4649 -----------------------
label: 1
score: 0
--------------------- 4650

--------------------- 4795 -----------------------
label: 1
score: 1.0
--------------------- 4796 -----------------------
label: 1
score: 0
--------------------- 4797 -----------------------
label: 4
score: 0
--------------------- 4798 -----------------------
label: 2
score: 0
--------------------- 4799 -----------------------
label: 1
score: 0
--------------------- 4800 -----------------------
label: 2
score: 0
--------------------- 4801 -----------------------
label: 1
score: 0.6666666666666666
--------------------- 4802 -----------------------
label: 4
score: 0.5
--------------------- 4803 -----------------------
label: 3
score: 0
--------------------- 4804 -----------------------
label: 2
score: 0
--------------------- 4805 -----------------------
label: 2
score: 0
--------------------- 4806 -----------------------
label: 1
score: 0
--------------------- 4807 -----------------------
label: 4
score: 0
--------------------- 4808 -----------------------
label: 1
score: 0
-------------

In [14]:
compare_4 = compare_df[compare_df['overall']==4.0]
compare_3 = compare_df[compare_df['overall']==3.0]
compare_2 = compare_df[compare_df['overall']==2.0]
compare_1 = compare_df[compare_df['overall']==1.0]
len(compare_df)
len(compare_4) + len(compare_3) + len(compare_2) +len(compare_1)

i_list = [1.0,2.0,3.0,4.0]
compare = [compare_1,compare_2,compare_3,compare_4]
for i in i_list:
    index = int(i)-1
    c = compare[index]
    print('Number of pairs that are rated as ',i,':')
    print(len(c))
    print('Pairs that are rated as ',i,', title named entity score is 0: (value, percentage)')
    print(len(c[c['title_ne_score']==0]), ', ', len(c[c['title_ne_score']==0])/len(c)*100,'%')
    print('Pairs that are rated as ',i,', title named entity score is > 0.3:')
    print(len(c[c['title_ne_score']>0.3]), ', ', len(c[c['title_ne_score']>0.3])/len(c)*100,'%')
    print('Pairs that are rated as ',i,', title named entity score is > 0.5:')
    print(len(c[c['title_ne_score']>0.5]), ', ', len(c[c['title_ne_score']>0.5])/len(c)*100,'%')
    print('Pairs that are rated as ',i,', title named entity score is 1.0:')
    print(len(c[c['title_ne_score']==1.0]), ', ', len(c[c['title_ne_score']==1.0])/len(c)*100,'%')

Number of pairs that are rated as  1.0 :
1443
Pairs that are rated as  1.0 , title named entity score is 0: (value, percentage)
930 ,  64.44906444906445 %
Pairs that are rated as  1.0 , title named entity score is > 0.3:
512 ,  35.48163548163548 %
Pairs that are rated as  1.0 , title named entity score is > 0.5:
380 ,  26.334026334026333 %
Pairs that are rated as  1.0 , title named entity score is 1.0:
362 ,  25.086625086625087 %
Number of pairs that are rated as  2.0 :
1124
Pairs that are rated as  2.0 , title named entity score is 0: (value, percentage)
792 ,  70.46263345195729 %
Pairs that are rated as  2.0 , title named entity score is > 0.3:
332 ,  29.537366548042705 %
Pairs that are rated as  2.0 , title named entity score is > 0.5:
227 ,  20.195729537366546 %
Pairs that are rated as  2.0 , title named entity score is 1.0:
220 ,  19.572953736654807 %
Number of pairs that are rated as  3.0 :
959
Pairs that are rated as  3.0 , title named entity score is 0: (value, percentage)
711 


### Remarks: <br>

As we expected, it's hard to tell if extracting NE from titles is a good feature to decide whether two articles are telling the same event. The reason being that some titles are NOT very descriptive, instead some tend to be sensationalistic or provocative, completely disregarding facts that we can extract NEs from. Even when two titles are associated to the same event, different news platforms may adopt different titling strategies; either they choose to keep it very short, or very long, or use an idiom, a phrase, a joke, or puns, making titling very much a stylistic choice. This is reasonable if one thinks that titles should be as striking and as different from one another as possible in order to be succesful. <br><br>

One argument against the use of title NEs as a feature is that  named entities appearing in the title are also likely be mentioned in the text. Since we are also extracting NEs from the text and comparing them in pairs, it can be argued that having title named entity similarity score is adding a similar feature that might not be necessary.

Nevertheless, from the previous short analysis, we can see a tendency when a pair is similar (rated as 1.0 or not much smaller), the high named entity similarity score occurs more common (25% of the title named entity have a score of 1.0 for the most similar pairs, and 8% of them are scored 1.0 for the least similar pairs (rated 4.0))<br><br>

To get a better judgement whether or not this feature is necessary, we will also examine the similarity score of the named entity extracted from the body text of the article (**insert DIRECTORY 17**).